In [3]:
import ccxt
import pandas as pd

In [4]:
def initialize_exchange():
    return ccxt.binanceusdm()

In [5]:
exchange = initialize_exchange()
exchange

ccxt.binanceusdm()

In [6]:
markets = exchange.fetch_markets()
markets[0]

{'id': 'BTCUSDT',
 'lowercaseId': 'btcusdt',
 'symbol': 'BTC/USDT:USDT',
 'base': 'BTC',
 'quote': 'USDT',
 'settle': 'USDT',
 'baseId': 'BTC',
 'quoteId': 'USDT',
 'settleId': 'USDT',
 'type': 'swap',
 'spot': False,
 'margin': False,
 'marginModes': {'cross': True, 'isolated': True},
 'swap': True,
 'future': False,
 'option': False,
 'active': True,
 'contract': True,
 'linear': True,
 'inverse': False,
 'taker': 0.0005,
 'maker': 0.0002,
 'contractSize': 1.0,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 0.001, 'price': 0.1, 'base': 1e-08, 'quote': 1e-08},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.001, 'max': 1000.0},
  'price': {'min': 556.8, 'max': 4529764.0},
  'cost': {'min': 100.0, 'max': None},
  'market': {'min': 0.001, 'max': 120.0}},
 'info': {'symbol': 'BTCUSDT',
  'pair': 'BTCUSDT',
  'contractType': 'PERPETUAL',
  'deliveryDate': 4133404800000,
  'onboardDate': 1569398400000,
  's

In [75]:
print(f"Volume : {int(exchange.fetch_ticker('BTC/USDT')['baseVolume'] * exchange.fetch_ticker('BTC/USDT')['last'] // 1_000_000):,} Billion USD")
print(int(exchange.fetch_ticker("BTC/USDT")["baseVolume"]))
print(int(exchange.fetch_ticker("BTC/USDT")["last"]))

Volume : 33,906 Billion USD
333630
101627


In [76]:
def fetch_price_data(exchange, symbols, since):
    price_data = {}
    for symbol in symbols:
        print(f"Fetching data for {symbol}")
        try:
            ohlcv = exchange.fetch_ohlcv(
                symbol, timeframe="1d", since=since, limit=1000
            )
            df = pd.DataFrame(
                ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
            )
            df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
            df.set_index("datetime", inplace=True)
            price_data[symbol] = df["close"]
        except Exception as e:
            print(f"Could not fetch data for {symbol}: {e}")
    return price_data

In [77]:
def categorize_symbols_by_correlation(correlations, thresholds):
    categorized = {f"Correlation >= {threshold}": [] for threshold in thresholds}

    for symbol, corr in correlations.items():
        for threshold in thresholds:
            if corr >= threshold:
                categorized[f"Correlation >= {threshold}"].append(symbol)
                break

    return categorized

In [78]:
def save_to_file(categorized_symbols, output_file):
    with open(output_file, "w") as file:
        for label, symbols in categorized_symbols.items():
            file.write(f"### {label}\n")
            formatted = ",".join(
                [f"BINANCE:{symbol.replace('/', '')}.P" for symbol in symbols]
            )
            file.write(formatted + "\n\n")
    print(f"Symbols saved to {output_file}")

In [91]:
exchange = initialize_exchange()
markets = exchange.fetch_markets()

markets = markets[:500]

usdt_symbols = []

for market in markets:
	if market["quote"] == "USDT":
		symbol = market["symbol"]
		try:
			ticker = exchange.fetch_ticker(symbol)
			if ticker and ticker.get("baseVolume") and ticker.get("last"):
				volume = int(ticker["baseVolume"] * ticker["last"] // 1_000_000)

				if volume > 20:
					# usdt_symbols.append({"Symbol": symbol[:-5], "Volume": volume})
					usdt_symbols.append(symbol[:-5])

		except Exception as e:
			print(f"Error fetching ticker for {symbol}: {e}")


since = exchange.parse8601("2023-01-01T00:00:00Z")
price_data = fetch_price_data(exchange, usdt_symbols, since)

prices_df = pd.DataFrame(price_data).dropna(axis=1, how="any")
returns_df = prices_df.pct_change().dropna()

btc_returns = returns_df["BTC/USDT"]
correlations = {
	symbol: returns_df[symbol].corr(btc_returns) for symbol in returns_df.columns
}

sorted_correlations = dict(
	sorted(correlations.items(), key=lambda item: item[1], reverse=True)
)

thresholds = [0.7, 0.65, 0.6]
categorized_symbols = categorize_symbols_by_correlation(
	sorted_correlations, thresholds
)

save_to_file(categorized_symbols, "TradingViewSymbols.txt")

Fetching data for BTC/USDT
Fetching data for ETH/USDT
Fetching data for BCH/USDT
Fetching data for XRP/USDT
Fetching data for EOS/USDT
Fetching data for LTC/USDT
Fetching data for TRX/USDT
Fetching data for ETC/USDT
Fetching data for LINK/USDT
Fetching data for XLM/USDT
Fetching data for ADA/USDT
Fetching data for XMR/USDT
Fetching data for DASH/USDT
Fetching data for ZEC/USDT
Fetching data for XTZ/USDT
Fetching data for BNB/USDT
Fetching data for ATOM/USDT
Fetching data for ONT/USDT
Fetching data for IOTA/USDT
Fetching data for BAT/USDT
Fetching data for VET/USDT
Fetching data for NEO/USDT
Fetching data for QTUM/USDT
Fetching data for IOST/USDT
Fetching data for THETA/USDT
Fetching data for ALGO/USDT
Fetching data for ZIL/USDT
Fetching data for KNC/USDT
Fetching data for ZRX/USDT
Fetching data for COMP/USDT
Fetching data for OMG/USDT
Fetching data for DOGE/USDT
Fetching data for SXP/USDT
Fetching data for KAVA/USDT
Fetching data for BAND/USDT
Fetching data for RLC/USDT
Fetching data f

/opt/homebrew/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/homebrew/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
